In [10]:
import pandas as pd
df1 = pd.read_csv(u'feature.csv')
df1.head()

,movieid,title,genres,string_agg,feature
0,3,天堂孤影,"['劇情,喜劇']",['Safka Pekkonen'],"天堂孤影 劇情,喜劇 卡德 羅娜 環衛 經理 服裝店 同事"
1,11,星際大戰四部曲：曙光乍現,"['科幻,冒險,動作']",['Alan Harris'],"星際大戰四部曲曙光乍現 科幻,冒險,動作 銀河 盧克 帝國 機器人 奧爾德 絕地"
2,12,海底總動員,"['家庭,動畫']",['Alexander Gould'],"海底總動員 家庭,動畫 馬林 尼莫 卵子 攻擊 潛水員 珊瑚"
3,13,阿甘正傳,"['愛情,劇情,喜劇']",['Steven Griffith'],"阿甘正傳 愛情,劇情,喜劇 阿甘 美國 一個 小鎮 愛護 尼克森"
4,14,美國心玫瑰情,['劇情'],['Peter Gallagher'],美國心玫瑰情 劇情 小珍 卡洛琳 於是 越來越 人生 罕住


In [11]:
import re
def sub(x):
    return (re.sub(r",", " ", x['feature']))

df1['feature'] = df1.apply(sub, axis = 1)

In [12]:
df1.head()

,movieid,title,genres,string_agg,feature
0,3,天堂孤影,"['劇情,喜劇']",['Safka Pekkonen'],天堂孤影 劇情 喜劇 卡德 羅娜 環衛 經理 服裝店 同事
1,11,星際大戰四部曲：曙光乍現,"['科幻,冒險,動作']",['Alan Harris'],星際大戰四部曲曙光乍現 科幻 冒險 動作 銀河 盧克 帝國 機器人 奧爾德 絕地
2,12,海底總動員,"['家庭,動畫']",['Alexander Gould'],海底總動員 家庭 動畫 馬林 尼莫 卵子 攻擊 潛水員 珊瑚
3,13,阿甘正傳,"['愛情,劇情,喜劇']",['Steven Griffith'],阿甘正傳 愛情 劇情 喜劇 阿甘 美國 一個 小鎮 愛護 尼克森
4,14,美國心玫瑰情,['劇情'],['Peter Gallagher'],美國心玫瑰情 劇情 小珍 卡洛琳 於是 越來越 人生 罕住


In [13]:
import gensim
from gensim.models import word2vec
from gensim.models.doc2vec import Doc2Vec
TaggededDocument = gensim.models.doc2vec.TaggedDocument
import pandas as pd
#import pymysql
import logging
import multiprocessing

words = []
for num,title in enumerate(df1['feature'].tolist()):
    title = [ str(x) for x in title.split() ]
    #document = TaggededDocument(title, tags=[num])
    words.append(title)

    
# print(words)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)   
model1= word2vec.Word2Vec(words, min_count=1, vector_size=100, epochs=5,workers=multiprocessing.cpu_count(), negative=5,sg=1)

#保存模型，供日後使用
model1.save("ENtest.model")

2021-11-15 15:22:11,889 : INFO : collecting all words and their counts
2021-11-15 15:22:11,890 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-11-15 15:22:11,915 : INFO : PROGRESS: at sentence #10000, processed 94723 words, keeping 28794 word types
2021-11-15 15:22:11,923 : INFO : collected 33907 word types from a corpus of 114358 raw words and 12160 sentences
2021-11-15 15:22:11,924 : INFO : Creating a fresh vocabulary
2021-11-15 15:22:12,083 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 33907 unique words (100.0%% of original 33907, drops 0)', 'datetime': '2021-11-15T15:22:12.083321', 'gensim': '4.0.1', 'python': '3.7.6 (default, Apr 24 2021, 01:03:20) \n[Clang 12.0.0 (clang-1200.0.32.29)]', 'platform': 'Darwin-20.5.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-11-15 15:22:12,084 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 114358 word corpus (100.0%% of original 114358, drops 0)', 'datetime'

In [14]:
##########電影item惟一貫量生成##############333
model1 = word2vec.Word2Vec.load("ENtest.model")
items = []
# for title in x_train_creative_id:
for num,title in enumerate(df1['feature'].tolist()):
    ss_product_id = []
    title1 = [ str(x) for x in title.split() ]
    for i in title1:
        ss_product_id.append(model1.wv[str(i)])
    ss = sum(ss_product_id)/len(ss_product_id)
 #     print(type(model_dm.infer_vector(title)))
    items.append(ss)


2021-11-15 15:22:13,414 : INFO : loading Word2Vec object from ENtest.model
2021-11-15 15:22:13,446 : INFO : loading wv recursively from ENtest.model.wv.* with mmap=None
2021-11-15 15:22:13,448 : INFO : setting ignored attribute cum_table to None
2021-11-15 15:22:13,795 : INFO : Word2Vec lifecycle event {'fname': 'ENtest.model', 'datetime': '2021-11-15T15:22:13.795431', 'gensim': '4.0.1', 'python': '3.7.6 (default, Apr 24 2021, 01:03:20) \n[Clang 12.0.0 (clang-1200.0.32.29)]', 'platform': 'Darwin-20.5.0-x86_64-i386-64bit', 'event': 'loaded'}


In [15]:
###########cos類似度推薦#######33
from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np


kk = cosine_similarity(items)
# for num,j in enumerate(kk):
#     item = df1['title'][num]
#     recs = []
#     for i in j.argsort()[::-1][1:10]:
#         recs.append(df1['title'][i])

#     print(item,recs)
#     print("#"*10)

In [16]:
# create list of indices for later matching
indices = pd.Series(df1.index, index = df1['title'])

In [17]:
user = pd.read_csv(u'userlist.csv')

df1.drop(['feature'],axis=1,inplace=True)
user = pd.merge(user, df1, on='movieid')
new = user.title.to_list()
user.head()

for i in new:
    print(i)

辛普森家庭電影版
海底總動員
魔法公主
回到未來


In [18]:
def recommend_movie(titles, n = 10, cosine_sim = kk):
    movies = []
    #for titles in userList
    for title in titles:
        # 檢索匹配的 movie_name index
        if title not in indices.index:
            print("Movie not in database.")
            return
        else:
            idx = indices[title]
        
        # 電影的餘弦相似度分數降序排列
        scores = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
            
        # 前 n 個最相似的 movies indexes
        # 使用 1:n 因為 0 是輸入的同一部電影
        top_n_idx = list(scores.iloc[1:n].index)
                
        #return result
        #print(df1['movie_name'].iloc[top_n_idx])
        ans = df1.iloc[top_n_idx]
        movies.append(ans)
    df = pd.concat(movies)    
    return df

In [19]:
ans = recommend_movie(new,7)
#ans
ans = ans.drop_duplicates(keep='first', inplace=False,subset=['title'])

In [20]:
ans.head()

,movieid,title,genres,string_agg
465,863,玩具總動員2,"['喜劇,動畫,家庭']",['Jess Harnell']
5673,234567,米奇拯救米妮大作戰,"['喜劇,動畫,家庭']",['Terri Douglas']
7810,411840,衝浪季節2：波浪狂熱,"['喜劇,家庭,動畫']",['Jon Heder']
7309,378236,表情符號電影,"['家庭,動畫,喜劇']",['Jennifer Coolidge']
1693,10258,馬達加斯加企鵝幫聖誕惡搞歷險記,"['家庭,動畫,喜劇']",['Rif Hutton']
